# Xử lý hình ảnh và văn bản với Mô hình Ngôn ngữ Thị giác (VLM)

Notebook này minh họa cách sử dụng mô hình `HuggingFaceTB/SmolVLM-Instruct` đã được lượng tử hóa **4 bit** cho các tác vụ đa phương thức (multimodal) khác nhau như:

- Hỏi đáp bằng hình ảnh (Visual Question Answering - VQA): Trả lời các câu hỏi dựa trên nội dung hình ảnh.

- Nhận dạng văn bản (OCR): Trích xuất và diễn giải văn bản trong hình ảnh.

- Mô tả video: Mô tả video thông qua phân tích khung hình tuần tự.

Bằng cách cấu trúc các lời nhắc (prompt) một cách hiệu quả, bạn có thể tận dụng mô hình cho nhiều ứng dụng, chẳng hạn như hiểu cảnh, phân tích tài liệu và suy luận hình ảnh động.

In [ ]:
# Cài đặt các yêu cầu trong Google Colab
# !pip install transformers datasets trl huggingface_hub bitsandbytes

# Xác thực với Hugging Face
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import torch, PIL
from transformers import AutoProcessor, AutoModelForVision2Seq, BitsAndBytesConfig
from transformers.image_utils import load_image

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

quantization_config = BitsAndBytesConfig(load_in_4bit=True)
model_name = "HuggingFaceTB/SmolVLM-Instruct"
model = AutoModelForVision2Seq.from_pretrained(
    model_name,
    quantization_config=quantization_config,
).to(device)
processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-Instruct")

print(processor.image_processor.size)

`low_cpu_mem_usage` was None, now default to True since model is quantized.
You shouldn't move a model that is dispatched using accelerate hooks.
Some kwargs in processor config are unused and will not have any effect: image_seq_len. 


{'longest_edge': 1536}


## Xử lý hình ảnh


Hãy bắt đầu với việc tạo chú thích và trả lời các câu hỏi về một hình ảnh. Chúng ta cũng sẽ khám phá việc xử lý nhiều hình ảnh.

### 1. Tạo chú thích cho một hình ảnh

In [3]:
from IPython.display import Image, display

image_url1 = "https://cdn.pixabay.com/photo/2024/11/20/09/14/christmas-9210799_1280.jpg"
display(Image(url=image_url1))

image_url2 = "https://cdn.pixabay.com/photo/2024/11/23/08/18/christmas-9218404_1280.jpg"
display(Image(url=image_url2))

In [4]:
# Tải một hình ảnh
image1 = load_image(image_url1)

# Tạo các thông báo đầu vào
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "Bạn có thể mô tả hình ảnh này không?"}
        ]
    },
]

# Chuẩn bị đầu vào
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image1], return_tensors="pt")
inputs = inputs.to(device)

# Tạo đầu ra
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)

print(generated_texts)

/home/duydl/Miniconda3/envs/py310/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


['User:<image>Can you describe the image?\nAssistant: The image is a scene of a person walking in a forest. The person is wearing a coat and a cap. The person is holding the hand of another person. The person is walking on a path. The path is covered with dry leaves. The background of the image is a forest with trees.']


### 2. So sánh nhiều hình ảnh
Mô hình có thể xử lý và so sánh nhiều hình ảnh. Hãy xác định chủ đề chung giữa hai hình ảnh.

In [5]:
# Tải hình ảnh
image2 = load_image(image_url2)

# Tạo các thông báo đầu vào
messages = [
    # {
    #     "role": "user",
    #     "content": [
    #         {"type": "image"},
    #         {"type": "image"},
    #         {"type": "text", "text": "Bạn có thể mô tả hai hình ảnh này không?"}
    #     ]
    # },
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "image"},
            {"type": "text", "text": "Chúng đại diện cho sự kiện gì?"}
        ]
    },
]

# Chuẩn bị đầu vào
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image1, image2], return_tensors="pt")
inputs = inputs.to(device)

# Tạo đầu ra
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)

print(generated_texts)

['User:<image>What event do they both represent?\nAssistant: Christmas.']


### 🔠 Nhận dạng văn bản (OCR)
VLM cũng có thể nhận ra và diễn giải văn bản trong hình ảnh, làm cho nó phù hợp với các tác vụ như phân tích tài liệu.
Bạn có thể thử nghiệm trên các hình ảnh có văn bản dày đặc hơn.

In [9]:
document_image_url = "https://cdn.pixabay.com/photo/2020/11/30/19/23/christmas-5792015_960_720.png"
display(Image(url=document_image_url))

# Tải hình ảnh tài liệu
document_image = load_image(document_image_url)

# Tạo thông báo đầu vào để phân tích
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "Nội dung là gì?"}
        ]
    }
]

# Chuẩn bị đầu vào
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[document_image], return_tensors="pt")
inputs = inputs.to(device)

# Tạo đầu ra
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)

print(generated_texts)

['User:<image>What is written?\nAssistant: MERRY CHRISTMAS AND A HAPPY NEW YEAR']


## Xử lý video
Các Mô hình Ngôn ngữ Thị giác (VLM) có thể xử lý video gián tiếp bằng cách trích xuất các khung hình chính (keyframe) và suy luận trên chúng theo thứ tự thời gian. Mặc dù VLM thiếu khả năng nhận biết thời gian thực của các mô hình video chuyên dụng, chúng vẫn có thể:

- Mô tả các hành động hoặc sự kiện bằng cách phân tích các khung hình được lấy mẫu tuần tự.

- Trả lời các câu hỏi về video dựa trên các khung hình chính đại diện.

- Tóm tắt nội dung video bằng cách kết hợp các mô tả văn bản của nhiều khung hình.

Hãy thử nghiệm trên một ví dụ:

<video width="640" height="360" controls>
  <source src="https://cdn.pixabay.com/video/2023/10/28/186794-879050032_large.mp4" type="video/mp4">
  Your browser does not support the video tag.
</video>

In [ ]:
# !pip install opencv-python

In [7]:
from IPython.display import Video
import cv2
import numpy as np

def extract_frames(video_path, max_frames=50, target_size=None):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Không thể mở video: {video_path}")
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = np.linspace(0, total_frames - 1, max_frames, dtype=int)

    frames = []
    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if ret:
            frame = PIL.Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            if target_size:
                frames.append(resize_and_crop(frame, target_size))
            else:
                frames.append(frame)
    cap.release()
    return frames

def resize_and_crop(image, target_size):
    width, height = image.size
    scale = target_size / min(width, height)
    image = image.resize((int(width * scale), int(height * scale)), PIL.Image.Resampling.LANCZOS)
    left = (image.width - target_size) // 2
    top = (image.height - target_size) // 2
    return image.crop((left, top, left + target_size, top + target_size))

# Đường dẫn video
video_link = "https://cdn.pixabay.com/video/2023/10/28/186794-879050032_large.mp4"

In [8]:
question = "Mô tả những gì người phụ nữ đang làm."

def generate_response(model, processor, frames, question):

    image_tokens = [{"type": "image"} for _ in frames]
    messages = [
        {
            "role": "user",
            "content": [{"type": "text", "text": "Sau đây là các khung hình của một video theo thứ tự thời gian."}, *image_tokens, {"type": "text", "text": question}]
        }
    ]
    inputs = processor(
        text=processor.apply_chat_template(messages, add_generation_prompt=True),
        images=frames,
        return_tensors="pt"
    ).to(model.device)

    outputs = model.generate(
        **inputs, max_new_tokens=100, num_beams=5, temperature=0.7, do_sample=True, use_cache=True
    )
    return processor.decode(outputs[0], skip_special_tokens=True)

# Trích xuất khung hình từ video
frames = extract_frames(video_link, max_frames=15, target_size=384)

processor.image_processor.size = (384, 384)
processor.image_processor.do_resize = False
# Tạo phản hồi
response = generate_response(model, processor, frames, question)

# Hiển thị kết quả
# print("Câu hỏi:", question)
print("Phản hồi:", response)

Response: User: Following are the frames of a video in temporal order.<image>Describe what the woman is doing.
Assistant: The woman is hanging an ornament on a Christmas tree.


## 💐 Bạn đã hoàn thành!
Notebook này đã minh họa cách sử dụng Mô hình Ngôn ngữ Thị giác (VLM) như định dạng lời nhắc cho các tác vụ đa phương thức. Bằng cách làm theo các bước được nêu ở đây, bạn có thể thử nghiệm với VLM và các ứng dụng của chúng.

### Các bước tiếp theo để khám phá:
- Thử nghiệm với nhiều trường hợp sử dụng VLM hơn.

- Cộng tác với đồng nghiệp bằng cách xem xét các yêu cầu kéo (PR) của họ.

- Đóng góp để cải thiện tài liệu khóa học này bằng cách mở một vấn đề (issue) hoặc gửi PR để giới thiệu các trường hợp sử dụng, ví dụ hoặc khái niệm mới.

Chúc bạn khám phá vui vẻ! 🌟